In [1]:
import csv
import requests
import os
import pandas as pd
import json
key = os.environ.get("ALPHA_VANTAGE_KEY")


In [2]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={}'.format(key)
print(CSV_URL)

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    #makeinto pandas dataframe, first row as header
    df = pd.DataFrame(cr)
    headers = df.iloc[0]
    df.columns = [headers]
    df.drop(index=0, axis=0, inplace=True)
    #create list of items as such { value: symbol, label: Name (Symbol) }
    items = []
    for index, row in df.iterrows():
        items.append({'value': row['name'] , 'label': row['symbol'] })
    #save list to json file
    with open('data.json', 'w') as outfile:
        json.dump(items, outfile)
    

https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=None


In [3]:
from fuzzywuzzy import fuzz

# Parse datajson to fuzzy match similarity with word and return the top 20 matches sorted by closeness match on value and word
def fuzzy(word):
    with open('data.json', 'r') as json_file:
        data = json.load(json_file)
        
        #by first letter of word and label
        dataload = list(filter(lambda x: x['label'].startswith(word[0].upper()), data))

        # Define a function to calculate the similarity score
        def similarity_score(item):
            # Modify this scoring as needed
            value_similarity = fuzz.token_sort_ratio(word.lower(), item['value'].lower())
            label_similarity = fuzz.token_sort_ratio(word.lower(), item['label'].lower())
            return value_similarity + label_similarity

        # Sort the data based on similarity score in descending order
        sorted_data = sorted(dataload, key=similarity_score, reverse=True)

        # Return the top 20 matches
        return sorted_data[:20]
    

fuzzy('tes')


c:\Users\User\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[{'value': 'Tenaris S.A.', 'label': 'TS'},
 {'value': 'Telos Corp', 'label': 'TLS'},
 {'value': 'Tesla Inc', 'label': 'TSLA'},
 {'value': 'TotalEnergies SE', 'label': 'TTE'},
 {'value': 'Terex Corp', 'label': 'TEX'},
 {'value': 'TSR Inc', 'label': 'TSRI'},
 {'value': 'Trinseo PLC', 'label': 'TSE'},
 {'value': 'Direxion Daily Technology Bear -3X Shares', 'label': 'TECS'},
 {'value': 'Teradyne Inc', 'label': 'TER'},
 {'value': 'Treehouse Foods Inc', 'label': 'THS'},
 {'value': 'Team Inc', 'label': 'TISI'},
 {'value': 'Tecnoglass Inc', 'label': 'TGLS'},
 {'value': 'Telefonica S.A', 'label': 'TEF'},
 {'value': 'Container Store Group Inc', 'label': 'TCS'},
 {'value': 'Tennessee Valley Authority', 'label': 'TVE'},
 {'value': 'Techprecision Corp', 'label': 'TPCS'},
 {'value': 'TREX Co. Inc', 'label': 'TREX'},
 {'value': 'Trimas Corporation', 'label': 'TRS'},
 {'value': 'Truecar Inc', 'label': 'TRUE'},
 {'value': 'Telus Corp', 'label': 'TU'}]

In [4]:
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
import numpy as np

In [5]:
key = os.environ.get("ALPHA_VANTAGE_KEY")
print(key)
CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={}'.format(key)
print(CSV_URL)

None
https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=None


In [8]:
!pip install yfinance

  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.0-py3-none-any.whl size=135766 sha256=2abcb4e9e49d6c407a8d59b4489b9b2679d1e9d171c9cf3d1a46f671b2c16d6a
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\77\6b\dc\722aee2a5d46a021ab67564c9d3d5536c3bd9577fcdc3a5aeb
Successfully built peewee
  Attempting uninstall: pytz
    Found existing installation: pytz 2021.3
    Uninstalling pytz-2021.3:
      Successfully uninstalled pytz-2021.3
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires multimethod>=1.4, which is not installed.
pandas-profiling 3.2.0 requires tangled-up-in-unicode==0.2.0, which is not installed.
pandas-profiling 3.2.0 requires visions[type_image_path]==0.7.4, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [14]:
stocks = ['MSFT', 'AAPL', 'TSLA']
market_index_ticker = '^GSPC'  # Replace with the market index ticker symbol (e.g., S&P 500)


# Set the date range for historical data (past year)
end_date = pd.to_datetime('today')
start_date = end_date - pd.DateOffset(years=1)


In [15]:
# Download historical prices using yfinance
stock_data = yf.download(stocks, start=start_date, end=end_date)['Adj Close']
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)['Adj Close']

# Calculate daily returns for stocks
stock_returns = stock_data.pct_change().dropna()

# Assume a constant risk-free rate for simplicity
risk_free_rate = 0.04

# Initialize lists to store CAPM metrics
betas = []
expected_returns = []

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  1 of 1 completed


In [13]:
stock_returns

,AAPL,MSFT,TSLA
Date,,,
2022-12-29,0.028324,0.027630,0.080827
2022-12-30,0.002469,-0.004938,0.011164
2023-01-03,-0.037405,-0.001001,-0.122422
2023-01-04,0.010314,-0.043743,0.051249
2023-01-05,-0.010605,-0.029638,-0.029039
...,...,...,...
2023-12-20,-0.010714,-0.007073,-0.039188
2023-12-21,-0.000770,0.007879,0.029781
2023-12-22,-0.005547,0.002784,-0.007701


In [17]:
# Calculate CAPM metrics for each stock
for stock in stocks:
    # Calculate beta using linear regression
    slope, _, _, _, _ = linregress(stock_returns[stock], stock_returns['^GSPC'])
    beta = slope

    # Estimate the expected market return (use historical average as an example)
    expected_market_return = stock_returns['^GSPC'].mean() * 252  # Assuming 252 trading days in a year

    # Calculate the market risk premium
    market_risk_premium = expected_market_return - risk_free_rate

    # Calculate expected return using CAPM formula
    expected_return = risk_free_rate + (beta * market_risk_premium)

    # Append values to lists
    betas.append(beta)
    expected_returns.append(expected_return)

KeyError: '^GSPC'